In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [4]:
def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

In [4]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [5]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [6]:
with open('../2021-12_demo/workflowD/parkinsons-crohns.json','r') as inf:
    query = json.load(inf)

In [7]:
printjson(query)

{
    "message": {
        "query_graph": {
            "edges": {
                "e00": {
                    "subject": "n00",
                    "object": "n01"
                },
                "e01": {
                    "subject": "n01",
                    "object": "n02"
                }
            },
            "nodes": {
                "n00": {
                    "ids": [
                        "DOID:14330"
                    ],
                    "categories": [
                        "biolink:Disease"
                    ]
                },
                "n01": {
                    "categories": [
                        "biolink:NamedThing"
                    ]
                },
                "n02": {
                    "ids": [
                        "DOID:8778"
                    ],
                    "categories": [
                        "biolink:Disease"
                    ]
                }
            }
        }
    }
}


In [9]:
result_id_1 = submit_to_ars(query)

https://arax.ncats.io/?source=ARS&id=32bc9435-404a-48b6-a196-2d94611fdaf4
